In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from IPython.display import display, HTML

/Users/nikitagupta/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
path1 = 'Correct_cities.csv'
path2 = 'Misspelt_cities.csv'
df_correct = pd.read_csv(path1)
df_incorrect = pd.read_csv(path2)

In [4]:
print(df_correct.dtypes)
print(df_incorrect.dtypes)

name       object
country    object
id          int64
dtype: object
misspelt_name    object
country          object
dtype: object


In [5]:
df_correct.head(2)

,name,country,id
0,les Escaldes,Andorra,3040051
1,Andorra la Vella,Andorra,3041563


In [6]:
df_incorrect.head(2)

,misspelt_name,country
0,Hfjdúszoposzló,Hungary
1,Otrajnyy,Russia


In [7]:
print(f'Count of records in correct cities - {df_correct.shape[0]}')
print(f'Count of unique countires in correct cities - {df_correct.country.nunique()}')
print(f'Count of unique cities in correct cities - {df_correct.name.nunique()}')
print(f'Count of unique id in correct cities - {df_correct.id.nunique()}')

print(f'\nCount of records in incorrect cities - {df_incorrect.shape[0]}')
print(f'Count of unique countires in incorrect cities - {df_incorrect.country.nunique()}')
print(f'Count of unique misspelt cities in incorrect countries - {df_incorrect.misspelt_name.nunique()}')

Count of records in correct cities - 23018
Count of unique countires in correct cities - 244
Count of unique cities in correct cities - 21940
Count of unique id in correct cities - 23018

Count of records in incorrect cities - 23018
Count of unique countires in incorrect cities - 244
Count of unique misspelt cities in incorrect countries - 22951


In [8]:
#Check is there exist any country in df_incorrect that is not present in correct cities dataframe
df_incorrect[~df_incorrect['country'].isin(df_correct['country'])]

,misspelt_name,country


In [9]:
#Check cities with same name belonging to different countries
df_correct_count = df_correct.groupby('name')['id'].count().reset_index().rename(columns={'id':'count_id'})
df_same_city_name = df_correct[df_correct['name'].isin(df_correct_count[df_correct_count['count_id']>1]['name'])].sort_values(by='name')
df_same_city_name.head()

,name,country,id
18731,Abadan,Turkmenistan,162099
11473,Abadan,Iran,145459
21584,Aberdeen,United States,5225857
7736,Aberdeen,United Kingdom,2657832
22274,Aberdeen,United States,5785243


In [10]:
def levenshtein(seq1, seq2):
    seq1 = seq1.lower()
    seq2 = seq2.lower()
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    return (matrix[size_x - 1, size_y - 1])


In [12]:
def calculate_score(incorrect_city_name, country):
    score_dict = {}
    df2_list = []
    df_check = df_correct[(df_correct['country']==country) & \
                          (df_correct['name'].str.len()==len(incorrect_city_name)) & \
                          (df_correct['selected']==False)]
    
    for correct_city in df_check.name.tolist():
        score = levenshtein(correct_city, incorrect_city_name)
        score_dict['score'] = score
        score_dict['correct_city'] = correct_city
        score_dict['incorrect_city'] = incorrect_city_name
        score_dict['country'] = country
        df2_list.append({'score': score, 'correct_city': correct_city, 'incorrect_city_name': incorrect_city_name, 'country':country})
    df_final = pd.DataFrame(df2_list).drop_duplicates()
    df_min_score = df_final[df_final['score']==df_final.score.min()]
    df_final = df_min_score.copy()
    if(df_min_score.shape[0]>1):
        df_min_score['ratio'] = df_min_score.apply(lambda x: get_jaro_distance(x['correct_city'],x['incorrect_city_name']), axis=1)
        df_final = df_min_score[df_min_score['ratio']==df_min_score['ratio'].max()]
        if (df_final.shape[0]>1):
            df_final = df_final.head(1)
    df_correct.loc[(df_correct['name']==df_final.iloc[0]['correct_city']) & (df_correct['country']==country),'selected']=True
    df_res = df_final.merge(df_correct, how='inner', left_on=['correct_city','country'], right_on=['name','country'])
    display(df_res[['incorrect_city_name','correct_city','country','id']])

In [13]:
df_test = df_incorrect.head(20)
df_correct['selected'] = False
ss = df_test.apply(lambda x: calculate_score(x['misspelt_name'], x['country']), axis=1)

,incorrect_city_name,correct_city,country,id
0,Hfjdúszoposzló,Hajdúszoboszló,Hungary,720276


,incorrect_city_name,correct_city,country,id
0,Otrajnyy,Otradnyy,Russia,513883


,incorrect_city_name,correct_city,country,id
0,ian Isidre,San Isidro,Peru,3929631


,incorrect_city_name,correct_city,country,id
0,Bordj Zemoufa,Bordj Zemoura,Algeria,2503620


,incorrect_city_name,correct_city,country,id
0,ChulamViwta,Chula Vista,United States,5336899


...

,incorrect_city_name,correct_city,country,id
0,Igbxho,Igboho,Nigeria,2338711


,incorrect_city_name,correct_city,country,id
0,Zibo,Zibo,China,1785286


,incorrect_city_name,correct_city,country,id
0,Shrīgoada,Shrīgonda,India,1256426


,incorrect_city_name,correct_city,country,id
0,Coatbricgu,Coatbridge,United Kingdom,2652696


,incorrect_city_name,correct_city,country,id
0,Arcmxko-Ekiti,Aramoko-Ekiti,Nigeria,2349529


#### Another Approach

In [20]:
def get_matches(query, country):
    df_check = df_correct[(df_correct['country']==country) & (df_correct['name'].str.len()==len(query))]
    
    results = process.extract(query, df_check['name'].tolist(), limit=3)
    df_res = pd.DataFrame(results, columns=['name','match'])
    res = df_res[df_res['match']==(df_res['match'].max())]

    res2 = df_check[df_check['name']==res['name'][0]]['id']
    return res2.values[0]

In [21]:
df_test = df_incorrect.copy()
df_test['match'] = df_test.apply(lambda x: get_matches(x['misspelt_name'], x['country']), axis=1)

In [22]:
df_incorrect_city_matches = df_test.merge(df_correct, how='inner', left_on=['country', 'match'], right_on=['country','id'])
df_incorrect_city_matches.head()

,misspelt_name,country,match,name,id
0,Hfjdúszoposzló,Hungary,720276,Hajdúszoboszló,720276
1,Otrajnyy,Russia,513883,Otradnyy,513883
2,ian Isidre,Peru,3929631,San Isidro,3929631
3,Bordj Zemoufa,Algeria,2503620,Bordj Zemoura,2503620
4,ChulamViwta,United States,5336899,Chula Vista,5336899


In [23]:
df_incorrect_city_matches.shape[0], df_test.shape[0], df_incorrect.shape[0]

(23018, 23018, 23018)